In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
os.chdir(os.path.join(os.path.dirname(os.getcwd()), 'data', 'in'))

In [ ]:
clientes_df = pd.read_csv('train_clientes_sample.csv')
requerimientos_df = pd.read_csv('train_requerimientos_sample.csv')

In [ ]:
def generar_variables_ingenieria(clientes_df):
    # SDO_ACTIVO (saldo)
    clientes_df["VAR_SDO_ACTIVO_6M"] = clientes_df["SDO_ACTIVO_MENOS0"] - clientes_df["SDO_ACTIVO_MENOS5"]
    clientes_df["PROM_SDO_ACTIVO_0M_2M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(3)]].mean(axis=1)
    clientes_df["PROM_SDO_ACTIVO_3M_5M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(3, 6)]].mean(axis=1)
    clientes_df["VAR_SDO_ACTIVO_3M"] = clientes_df["PROM_SDO_ACTIVO_0M_2M"] - clientes_df["PROM_SDO_ACTIVO_3M_5M"]
    clientes_df["PROM_SDO_ACTIVO_6M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(6)]].mean(axis=1)

    # FLG_SEGURO (flag binario)
    clientes_df["MESES_CON_SEGURO"] = clientes_df[[f"FLG_SEGURO_MENOS{i}" for i in range(6)]].sum(axis=1)

    # CANALES
    for canal in [1, 2, 3]:
        base = f"NRO_ACCES_CANAL{canal}_MENOS"
        clientes_df[f"VAR_NRO_ACCES_CANAL{canal}_6M"] = clientes_df[f"{base}0"] - clientes_df[f"{base}5"]
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_6M"] = clientes_df[[f"{base}{i}" for i in range(6)]].mean(axis=1)
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_0M_2M"] = clientes_df[[f"{base}{i}" for i in range(3)]].mean(axis=1)
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_3M_5M"] = clientes_df[[f"{base}{i}" for i in range(3, 6)]].mean(axis=1)
        clientes_df[f"VAR_NRO_ACCES_CANAL{canal}_3M"] = (clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_0M_2M"] - clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_3M_5M"])

    # ENTIDADES FINANCIERAS
    clientes_df["PROM_NRO_ENTID_SSFF_6M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(6)]].mean(axis=1)
    clientes_df["VAR_NRO_ENTID_SSFF_6M"] = clientes_df["NRO_ENTID_SSFF_MENOS0"] - clientes_df["NRO_ENTID_SSFF_MENOS5"]
    clientes_df["PROM_NRO_ENTID_SSFF_0M_2M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(3)]].mean(axis=1)
    clientes_df["PROM_NRO_ENTID_SSFF_3M_5M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(3, 6)]].mean(axis=1)
    clientes_df["VAR_NRO_ENTID_SSFF_3M"] = (clientes_df["PROM_NRO_ENTID_SSFF_0M_2M"] - clientes_df["PROM_NRO_ENTID_SSFF_3M_5M"])

    # SALDO EN OTRAS ENTIDADES
    clientes_df["MESES_CON_SALDO"] = clientes_df[[f"FLG_SDO_OTSSFF_MENOS{i}" for i in range(6)]].sum(axis=1)

    return clientes_df


In [ ]:
def imputacion_variables(clientes_df,requerimientos_df):
    moda = clientes_df['RANG_INGRESO'].mode()[0]
    clientes_df['RANG_INGRESO'].fillna(moda, inplace=True)

    moda = clientes_df['FLAG_LIMA_PROVINCIA'].mode()[0]
    clientes_df['FLAG_LIMA_PROVINCIA'].fillna(moda, inplace=True)

    clientes_df['EDAD'].fillna(clientes_df['EDAD'].median(), inplace=True)
    clientes_df['ANTIGUEDAD'].fillna(clientes_df['ANTIGUEDAD'].median(), inplace=True)

    moda = requerimientos_df['DICTAMEN'].mode()[0]
    requerimientos_df['DICTAMEN'].fillna(moda, inplace=True)

    return clientes_df, requerimientos_df

In [ ]:
def encoder_categoricos(clientes_df):
    clientes_df['RANG_SDO_PASIVO_MENOS0'] = clientes_df['RANG_SDO_PASIVO_MENOS0'].replace('Cero', 'Rango_SDO_00')
    clientes_df['FLAG_LIMA_PROVINCIA'] = clientes_df['FLAG_LIMA_PROVINCIA'].map({'Lima': 1, 'Provincia': 0})
    cat_cols = clientes_df.select_dtypes(include=['object', 'category']).columns
    encoders_clientes = {} 

    for col in cat_cols:
        le = LabelEncoder()
        clientes_df[col] = le.fit_transform(clientes_df[col])
        encoders_clientes[col] = le

    return clientes_df, encoders_clientes

In [ ]:
def construir_variables_requerimientos(df_reqs, id_col='ID_CORRELATIVO'):
    
    total_reqs = df_reqs.groupby(id_col).size().rename('total_requerimientos')
    if not isinstance(total_reqs, pd.DataFrame):
        total_reqs = total_reqs.to_frame()

    n_tipo_req = df_reqs.groupby(id_col)['TIPO_REQUERIMIENTO2'].nunique().rename('nro_tipos_requerimiento').to_frame()
    n_dictamen = df_reqs.groupby(id_col)['DICTAMEN'].nunique().rename('nro_dictamenes').to_frame()
    n_producto = df_reqs.groupby(id_col)['PRODUCTO_SERVICIO_2'].nunique().rename('nro_productos_servicios').to_frame()
    n_submotivo = df_reqs.groupby(id_col)['SUBMOTIVO_2'].nunique().rename('nro_submotivos').to_frame()

    tipo_ohe = pd.get_dummies(df_reqs['TIPO_REQUERIMIENTO2'], prefix='tipo')
    tipo_ohe[id_col] = df_reqs[id_col]
    tipo_ohe = tipo_ohe.groupby(id_col).sum()

    dictamen_ohe = pd.get_dummies(df_reqs['DICTAMEN'], prefix='dictamen')
    dictamen_ohe[id_col] = df_reqs[id_col]
    dictamen_ohe = dictamen_ohe.groupby(id_col).sum()

    df_agregado = pd.concat([total_reqs, n_tipo_req, n_dictamen, n_producto, n_submotivo, tipo_ohe, dictamen_ohe],axis=1)

    return df_agregado


In [ ]:
def estandarizacion(df_final):
    no_escalar = ['ID_CORRELATIVO', 'CODMES', 'ATTRITION']

    columnas_a_escalar = df_final.columns.difference(no_escalar)

    df_predictoras = df_final[columnas_a_escalar]

    scaler = StandardScaler()
    df_escaladas = pd.DataFrame(scaler.fit_transform(df_predictoras),columns=columnas_a_escalar,index=df_final.index)

    df_final_estandarizado = pd.concat([df_final[no_escalar], df_escaladas],axis=1)

    return df_final_estandarizado, scaler

In [ ]:
clientes_df = generar_variables_ingenieria(clientes_df)
clientes_df,requerimientos_df = imputacion_variables(clientes_df,requerimientos_df)
clientes_df, artifact_encoders_clientes = encoder_categoricos(clientes_df)
requerimientos_df = construir_variables_requerimientos(requerimientos_df)
df_final = clientes_df.merge(requerimientos_df, on='ID_CORRELATIVO', how='inner')
df_final, artifact_scaler = estandarizacion(df_final)

In [ ]:
df_final.columns

### test

In [2]:
os.chdir(os.path.join(os.path.dirname(os.getcwd()), 'data', 'out'))

In [3]:
os.getcwd()

'c:\\Users\\Omen\\UP\\Teoria importante\\ML OPS\\Proyecto final\\Bank_Attrition_Detection_MLOps\\data\\out'

In [4]:
df_data_test = pd.read_csv("clientes_data_test.csv")
df_requerimientos_test = pd.read_csv("requerimientos_data_test.csv")

In [5]:
os.chdir('../..')

In [6]:
os.chdir('outputs/preprocess')

In [7]:
os.getcwd()

'c:\\Users\\Omen\\UP\\Teoria importante\\ML OPS\\Proyecto final\\Bank_Attrition_Detection_MLOps\\outputs\\preprocess'

In [8]:
def prepare_impute_missing(df_data, x_cols):
    df_data_imputed = df_data.copy()
    df_impute_parameters = pd.read_csv(f"imputacion_parametros.csv")
    for col in x_cols:
        impute_value = df_impute_parameters[df_impute_parameters["variable"]==col]["valor"].values[0]
        df_data_imputed[col] = df_data_imputed[col].fillna(impute_value)
    return df_data_imputed

In [9]:
def generar_variables_ingenieria(clientes_df):
    clientes_df["VAR_SDO_ACTIVO_6M"] = clientes_df["SDO_ACTIVO_MENOS0"] - clientes_df["SDO_ACTIVO_MENOS5"]
    clientes_df["PROM_SDO_ACTIVO_0M_2M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(3)]].mean(axis=1)
    clientes_df["PROM_SDO_ACTIVO_3M_5M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(3, 6)]].mean(axis=1)
    clientes_df["VAR_SDO_ACTIVO_3M"] = clientes_df["PROM_SDO_ACTIVO_0M_2M"] - clientes_df["PROM_SDO_ACTIVO_3M_5M"]
    clientes_df["PROM_SDO_ACTIVO_6M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(6)]].mean(axis=1)
    clientes_df["MESES_CON_SEGURO"] = clientes_df[[f"FLG_SEGURO_MENOS{i}" for i in range(6)]].sum(axis=1)
    for canal in [1, 2, 3]:
        base = f"NRO_ACCES_CANAL{canal}_MENOS"
        clientes_df[f"VAR_NRO_ACCES_CANAL{canal}_6M"] = clientes_df[f"{base}0"] - clientes_df[f"{base}5"]
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_6M"] = clientes_df[[f"{base}{i}" for i in range(6)]].mean(axis=1)
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_0M_2M"] = clientes_df[[f"{base}{i}" for i in range(3)]].mean(axis=1)
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_3M_5M"] = clientes_df[[f"{base}{i}" for i in range(3, 6)]].mean(axis=1)
        clientes_df[f"VAR_NRO_ACCES_CANAL{canal}_3M"] = (clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_0M_2M"] - clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_3M_5M"])
    clientes_df["PROM_NRO_ENTID_SSFF_6M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(6)]].mean(axis=1)
    clientes_df["VAR_NRO_ENTID_SSFF_6M"] = clientes_df["NRO_ENTID_SSFF_MENOS0"] - clientes_df["NRO_ENTID_SSFF_MENOS5"]
    clientes_df["PROM_NRO_ENTID_SSFF_0M_2M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(3)]].mean(axis=1)
    clientes_df["PROM_NRO_ENTID_SSFF_3M_5M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(3, 6)]].mean(axis=1)
    clientes_df["VAR_NRO_ENTID_SSFF_3M"] = (clientes_df["PROM_NRO_ENTID_SSFF_0M_2M"] - clientes_df["PROM_NRO_ENTID_SSFF_3M_5M"])
    clientes_df["MESES_CON_SALDO"] = clientes_df[[f"FLG_SDO_OTSSFF_MENOS{i}" for i in range(6)]].sum(axis=1)

    return clientes_df

In [10]:
def construir_variables_requerimientos(df_reqs, id_col='ID_CORRELATIVO'):
        total_reqs = df_reqs.groupby(id_col).size().rename('total_requerimientos')
        if not isinstance(total_reqs, pd.DataFrame):
            total_reqs = total_reqs.to_frame()
        n_tipo_req = df_reqs.groupby(id_col)['TIPO_REQUERIMIENTO2'].nunique().rename('nro_tipos_requerimiento').to_frame()
        n_dictamen = df_reqs.groupby(id_col)['DICTAMEN'].nunique().rename('nro_dictamenes').to_frame()
        n_producto = df_reqs.groupby(id_col)['PRODUCTO_SERVICIO_2'].nunique().rename('nro_productos_servicios').to_frame()
        n_submotivo = df_reqs.groupby(id_col)['SUBMOTIVO_2'].nunique().rename('nro_submotivos').to_frame()
        tipo_ohe = pd.get_dummies(df_reqs['TIPO_REQUERIMIENTO2'], prefix='tipo')
        tipo_ohe[id_col] = df_reqs[id_col]
        tipo_ohe = tipo_ohe.groupby(id_col).sum()
        dictamen_ohe = pd.get_dummies(df_reqs['DICTAMEN'], prefix='dictamen')
        dictamen_ohe[id_col] = df_reqs[id_col]
        dictamen_ohe = dictamen_ohe.groupby(id_col).sum()
        df_agregado = pd.concat([total_reqs, n_tipo_req, n_dictamen, n_producto, n_submotivo, tipo_ohe, dictamen_ohe],axis=1)
        return df_agregado

In [11]:
def apply_label_encoders_to_test(df_test):
        df_test['RANG_SDO_PASIVO_MENOS0'] = df_test['RANG_SDO_PASIVO_MENOS0'].replace('Cero', 'Rango_SDO_00')
        df_test['FLAG_LIMA_PROVINCIA'] = df_test['FLAG_LIMA_PROVINCIA'].map({'Lima': 1, 'Provincia': 0})
        path_encoder='label_encoder_train.pkl'
        with open(path_encoder, 'rb') as f:
            encoders_clientes = pickle.load(f)
        for col, le in encoders_clientes.items():
            df_test[col] = le.transform(df_test[col])
        return df_test

In [12]:
def aplicar_estandarizacion_test(df_test):
    path_scaler='scaler_train.pkl'
    with open(path_scaler, 'rb') as f:
        scaler = pickle.load(f)
    no_escalar = ['ID_CORRELATIVO', 'CODMES', 'ATTRITION']
    columnas_a_escalar = df_test.columns.difference(no_escalar)
    df_predictoras = df_test[columnas_a_escalar]
    df_escaladas = pd.DataFrame(scaler.transform(df_predictoras),columns=columnas_a_escalar,index=df_test.index)
    df_test_estandarizado = pd.concat([df_test[no_escalar], df_escaladas], axis=1)
    return df_test_estandarizado

In [13]:
def prepare_dataset(df_data_test,df_requerimientos_test):
    x_cols_clientes = ['RANG_INGRESO','FLAG_LIMA_PROVINCIA','EDAD','ANTIGUEDAD']
    x_cols_requerimientos = ['DICTAMEN']
    df_data_imputed_clientes = prepare_impute_missing(df_data_test, x_cols_clientes)
    df_data_imputed_requerimientos = prepare_impute_missing(df_requerimientos_test, x_cols_requerimientos)
    df_data_feature_clientes = generar_variables_ingenieria(df_data_imputed_clientes)
    df_data_feature_requerimientos = construir_variables_requerimientos(df_data_imputed_requerimientos)
    df_data_encoder_clientes = apply_label_encoders_to_test(df_data_feature_clientes)
    df_final = df_data_encoder_clientes.merge(df_data_feature_requerimientos, on='ID_CORRELATIVO', how='inner')
    df_final = aplicar_estandarizacion_test(df_final)
    return df_final

In [14]:
df_final = prepare_dataset(df_data_test,df_requerimientos_test)

In [17]:
df_final.columns

Index(['ID_CORRELATIVO', 'CODMES', 'ATTRITION', 'ANTIGUEDAD', 'EDAD',
       'FLAG_LIMA_PROVINCIA', 'FLG_BANCARIZADO', 'FLG_NOMINA',
       'FLG_SDO_OTSSFF_MENOS0', 'FLG_SDO_OTSSFF_MENOS1',
       'FLG_SDO_OTSSFF_MENOS2', 'FLG_SDO_OTSSFF_MENOS3',
       'FLG_SDO_OTSSFF_MENOS4', 'FLG_SDO_OTSSFF_MENOS5', 'FLG_SEGURO_MENOS0',
       'FLG_SEGURO_MENOS1', 'FLG_SEGURO_MENOS2', 'FLG_SEGURO_MENOS3',
       'FLG_SEGURO_MENOS4', 'FLG_SEGURO_MENOS5', 'MESES_CON_SALDO',
       'MESES_CON_SEGURO', 'NRO_ACCES_CANAL1_MENOS0',
       'NRO_ACCES_CANAL1_MENOS1', 'NRO_ACCES_CANAL1_MENOS2',
       'NRO_ACCES_CANAL1_MENOS3', 'NRO_ACCES_CANAL1_MENOS4',
       'NRO_ACCES_CANAL1_MENOS5', 'NRO_ACCES_CANAL2_MENOS0',
       'NRO_ACCES_CANAL2_MENOS1', 'NRO_ACCES_CANAL2_MENOS2',
       'NRO_ACCES_CANAL2_MENOS3', 'NRO_ACCES_CANAL2_MENOS4',
       'NRO_ACCES_CANAL2_MENOS5', 'NRO_ACCES_CANAL3_MENOS0',
       'NRO_ACCES_CANAL3_MENOS1', 'NRO_ACCES_CANAL3_MENOS2',
       'NRO_ACCES_CANAL3_MENOS3', 'NRO_ACCES_CANAL3_MENO